## encode movies

In [ ]:
#   drop duplicates
import pandas as pd
from Levenshtein import distance

df = pd.read_csv('mpst_raw.csv')
#only movie titles
df = df.sort_values('title').reset_index(drop=True)
df['lev'] = None
df

for a in range(len(df)-1):
    if distance(df.iloc[a].title, df.iloc[a+1].title) <= 3:
        print(a, df.iloc[a].title, df.iloc[a+1].title)
        df.at[a, 'lev'] = distance(df.iloc[a].title, df.iloc[a+1].title)
df
#we filter similar movies
df = df[df['lev'].isnull()].reset_index(drop=True)
df

#find Avengers duplicates
for a in range(len(df)):
    if df.iloc[a]['title'].find('Avengers') != -1:
        pass
        #print(a)
#drop extra
df = df.drop([9572]).reset_index(drop=True) #i can do 1, 2, 3... to drop multiple
df
df.to_csv('mpst_drop.csv')

In [ ]:
#   encode
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import numpy as np

tqdm.pandas()

model = SentenceTransformer('all-MiniLM-L6-v2') #all-MiniLM-L6-v2 #all-mpnet-base-v2

df_ = df.copy()
df_['plot_synopsis'] = df_['plot_synopsis'].progress_apply(lambda x : model.encode(x))
df_index = df_.pop('title')
df_ = df_[['plot_synopsis']]
df_ = pd.DataFrame(np.column_stack(list(zip(*df_.values))))
df_.index = df_index
df_.to_csv('mpst_drop_encoded.csv')

## encode books

In [ ]:
#   drop duplicates
import pandas as pd

df = pd.read_csv('GoodReads_100k_books.csv')
df = df.dropna()
df = df[['title', 'desc']].drop_duplicates('title')
df['title'] = df['title'].apply(lambda x: x.lower())
df = df.reset_index(drop=True)
df

In [ ]:
#   encode
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import numpy as np

tqdm.pandas()

model = SentenceTransformer('all-MiniLM-L6-v2') #all-MiniLM-L6-v2 #all-mpnet-base-v2

def encode_desc(df, index_col, description_col):
    df_ = df.copy()
    df_[description_col] = df_[description_col].progress_apply(lambda x : model.encode(x))
    df_index = df_.pop(index_col)
    df_ = df_[[description_col]]
    df_ = pd.DataFrame(np.column_stack(list(zip(*df_.values))))
    df_.index = df_index
    return df_

df = encode_desc(df, 'title', 'desc')
df.to_csv('GoodReads_drop_encoded.csv')

## add movies manually

In [ ]:
#upload df
import pandas as pd

#df_books_raw = pd.read_csv('GoodReads_drop.csv').drop('Unnamed: 0', axis=1)
#df_books_raw.index = df_books_raw.pop('title')

df_movies_encoded = pd.read_csv('mpst_drop_encoded.csv')
df_movies_encoded.index = df_movies_encoded.pop('title')
df_movies_encoded

In [59]:
#df with the data to add
plot1 = """
The story starts off with the focus on Seong Gi-hun (Lee Jung-jae), a lazy but well-meaning man who’s living on the back of his elderly mother’s meagre income. Gi-hun is a divorcee with a young daughter who he has made a habit of disappointing.
He’s knee-deep in debt and uses up the money his mother lends him to feed a troublesome betting habit. He lucks out at the track and wins a lot of money but he runs into some debt collectors and while trying to get away from them, he’s pickpocketed by a young girl. The collectors catch up with him and make him sign a physical contract promising his organs in case of any more delays in payment.
That night at a subway station, he meets a well-suited gentleman who offers to play a fairly simple children’s game with a catch. If Gi-hun wins a round, he’ll be paid 100,000 won, but if he loses a round, the gentleman slaps him in return.
After multiple rounds of failure, Gi-hun finally wins one and is given the money. Along with that, he’s given a card with a phone number, an offer to join a game that promises to be the solution to the end of all his problems. After much contemplation, he calls the number and agrees to take part.
The people who agreed to play are picked up and taken to an undisclosed location. They all wake up in a room with beds and wearing numbered jumpsuits. A group of men in masks come to explain that they’ll have to play 6 games and win them all in order to get the prize money. When one contestant questions why they should play after they were drugged and had their belongings taken from them, the masked men remind the players that each and every one of them is under a lot of debt and this game is key to their survival.
At the venue, Gi-hun meets his childhood friend Cho Sang-woo (Park Hae-soo) who’s supposed to be a university graduate and currently abroad on a business trip. It is revealed that he is actually charged with fraud and embezzlement, which is why he finds himself in this predicament.
The first game they have to play is ‘Red light, Green light’. However, in a horrible twist, if a player is caught in this game, elimination equates to death. Everyone starts panicking and is subsequently shot down until the remaining players decide to actually play the game. By the end, almost half of all the players are eliminated with the rest in shock regarding what they have signed on for.
Bringing up one of the rules of the contract, they decide to have a vote to leave the game. The Majority vote for the game to end and they’re all released with the offer to return and continue the game still open. After spending a few days outside and realizing that their lives are no better, most of the contestants decide to return. This time, a detective named Hwang Jun-ho (Wi Ha-joon), manages to sneak aboard the ship to the venue in search of his missing brother.
The next game they’re made to play is sugar honeycombs. The players have to carve out a particular shape from a biscuit without breaking it. Thanks to a hint from Kang Se-byeok (Jung Ho-yeon) who had sneaked through the vents earlier, Sang-woo guesses what the game is but doesn’t share that knowledge with Gi-hun. Despite getting the umbrella shape, Gi-hun figures out a trick to make things easier and survives the second game as well.
Alliances begin to form among the players and after an altercation leads to the death of a player with no real consequences, a riot breaks out at night to thin out the herd. The third game contested is the Tug-of-War, and despite having a weaker team on paper, Gi-hun and his group survive thanks to the sage advice of the elderly Oh Il-nam (Oh Young-soo) and some quick thinking by Sang-woo.
The masked men instruct the players to make teams of two for the next game which turns out to be marbles. The twist is that players in each team actually play each other. Sang-woo joins up the simple Ali Abdul (Tripati Anupam) while Gi-hun teams up with Il-nam. Sang-woo is close to losing when he decides to trick Ali into giving him all the marbles, Gi-hun takes advantage of Il-nam’s dementia to get through to the next round.
A group of wealthy VIP’s arrive at the island to witness the final two games in person, betting on the competitors to make things more interesting. On the outside, a search is carried out for Jun-ho, who’s running his own investigation to find out the truth behind this sick game.
The fifth game is revealed to be glass stepping stones. With two adjacent bridges, the players have to choose between a normal glass or a tempered glass step. The wrong choice means the glass will shatter under their weight, sending them down to their doom. Players sabotage each other to get ahead with only three of them getting to the end, Sang-woo, Gi hun and Se-byeok. At the end of the game, the remaining glass steps shatter in an explosion with one shard stabbing Se-byeok, wounding her.
Before the final game, the three contestants are treated to a fancy dinner and at the end of the dinner, they are encouraged to carry along their steak knives with them. Gi-hun sticks with Se-byeok in case Sang-woo decides to attack but when she collapses due to excessive blood loss, he rushes to the door for help. During this time, Sang-woo seizes the opportunity to land the final blow and bring it down to the two players.
"""

In [94]:
new_movies = pd.DataFrame(['The squid game', plot1.replace('\n', '')]).T
new_movies.columns = ['title', 'desc']
new_movies

,title,desc
0,The squid game,The story starts off with the focus on Seong G...


In [95]:
#encoding
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import numpy as np

tqdm.pandas()

model = SentenceTransformer('all-MiniLM-L6-v2') #all-MiniLM-L6-v2 #all-mpnet-base-v2

def encode_desc(df, index_col, description_col):
    df_ = df.copy()
    df_[description_col] = df_[description_col].progress_apply(lambda x : model.encode(x))
    df_index = df_.pop(index_col)
    df_ = df_[[description_col]]
    df_ = pd.DataFrame(np.column_stack(list(zip(*df_.values))))
    df_.index = df_index
    return df_

df_new = encode_desc(new_movies, 'title', 'desc')
df_new.columns = [str(x) for x in list(df_new.columns)]
df_new
#df.to_csv('GoodReads_encoded_no_duplicates.csv')

100%|██████████| 1/1 [00:00<00:00,  7.27it/s]


,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
title,,,,,,,,,,,,,,,,,,,,,
The squid game,-0.153858,0.069498,-0.039437,0.007709,-0.057025,-0.005656,0.147098,0.032641,0.013569,-0.006615,...,0.037843,-0.108261,0.054256,0.030857,-0.035345,-0.015266,0.064503,-0.065952,-0.029446,-0.001071


In [96]:
df_movies_encoded = pd.concat([df_movies_encoded, df_new], axis=0)
df_movies_encoded

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
title,,,,,,,,,,,,,,,,,,,,,
$,-0.024897,0.055245,-0.122327,-0.029603,-0.007433,0.036262,0.129328,0.020940,0.048176,-0.037479,...,0.032808,-0.081902,-0.002921,0.033075,0.036488,0.016339,0.081623,-0.065515,0.016297,-0.047112
$windle,-0.032736,0.183432,-0.103036,-0.038543,-0.011393,0.050009,0.066918,0.036443,0.018850,0.002050,...,0.012507,-0.092158,-0.028933,-0.058818,0.019628,-0.001966,0.089710,0.011076,-0.016599,0.075246
'71,-0.080760,0.002873,-0.099334,-0.058721,0.020339,0.047407,0.044057,-0.010748,-0.049804,0.035990,...,-0.013720,-0.032820,0.007075,0.006609,-0.014338,0.049984,-0.021645,-0.003516,-0.011251,-0.024905
'A' gai wak,-0.114455,0.049236,0.034111,0.006510,-0.012836,0.074500,0.064006,-0.022156,-0.054337,0.011865,...,-0.042240,0.089022,0.016201,0.042965,-0.108093,0.070876,0.028978,-0.021293,-0.020478,0.043706
'Breaker' Morant,-0.114930,0.037942,-0.024508,-0.044298,0.029797,0.022180,0.019458,0.065945,0.001178,0.091680,...,0.025499,0.035366,0.037543,-0.014330,-0.054029,0.015875,-0.022152,-0.068856,-0.022993,-0.009707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ådalen 31,-0.070786,0.102453,-0.025706,0.000993,0.014982,0.061297,0.107749,-0.013328,-0.007382,0.016728,...,0.006848,-0.029199,0.007583,-0.063745,-0.061957,0.053530,0.007582,0.002616,-0.015631,0.048152
Él,-0.032808,0.009943,0.047571,0.073746,0.043413,0.080470,-0.037416,0.024547,0.134191,-0.069619,...,0.038110,-0.026639,-0.023677,0.021118,0.026846,0.043454,-0.043832,0.004737,-0.037642,-0.059959
Ôdishon,-0.008681,-0.068630,-0.018248,0.010679,-0.005883,0.033397,0.035146,-0.007881,-0.012534,0.011920,...,-0.106348,-0.029262,-0.052140,-0.035167,0.040802,0.064893,0.135530,-0.004852,-0.008708,0.006679


In [97]:
#add it to the old df and replace current .csv file
df_movies_encoded.to_csv('mpst_drop_encoded+.csv')

## recommendation system movie -> books

In [ ]:
from fuzzywuzzy import fuzz, process

#cosa cazzo fa?
pd.DataFrame(process.extractBests('Price of Egypt', choices=df_movies_encoded.index, limit=10, score_cutoff=0)).sort_values(1, ascending=False)

In [ ]:
from Levenshtein import distance

#string-searching algorithm
m = pd.DataFrame(df_movies_encoded.index)
m['lev'] = m['title'].apply(lambda x : distance(x, 'Price of Egypt'))
m.sort_values('lev', ascending=True)

In [98]:
import pandas as pd

df_books_raw = pd.read_csv('GoodReads_no_duplicates.csv').drop('Unnamed: 0', axis=1)
df_books_raw.index = df_books_raw.pop('title')

df_movies_raw = pd.read_csv('mpst_no_duplicates.csv').drop('Unnamed: 0', axis=1)
df_movies_raw.index = df_movies_raw.pop('title')

C:\Users\ardit\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [99]:
df_books_encoded = pd.read_csv('GoodReads_encoded_no_duplicates.csv')
df_movies_encoded = pd.read_csv('mpst_encoded_no_duplicates_new.csv')

df_movies_encoded['label'] = 'movie'
df_books_encoded['label'] = 'book'

df_movies_encoded.index = df_movies_encoded.pop('title')
df_books_encoded.index = df_books_encoded.pop('title')

In [100]:
df = pd.concat([df_movies_encoded, df_books_encoded], axis=0)
df

,0,1,2,3,4,5,6,7,8,9,...,375,376,377,378,379,380,381,382,383,label
title,,,,,,,,,,,,,,,,,,,,,
$,-0.024897,0.055245,-0.122327,-0.029603,-0.007433,0.036262,0.129328,0.020940,0.048176,-0.037479,...,-0.081902,-0.002921,0.033075,0.036488,0.016339,0.081623,-0.065515,0.016297,-0.047112,movie
$windle,-0.032736,0.183432,-0.103036,-0.038543,-0.011393,0.050009,0.066918,0.036443,0.018850,0.002050,...,-0.092158,-0.028933,-0.058818,0.019628,-0.001966,0.089710,0.011076,-0.016599,0.075246,movie
'71,-0.080760,0.002873,-0.099334,-0.058721,0.020339,0.047407,0.044057,-0.010748,-0.049804,0.035990,...,-0.032820,0.007075,0.006609,-0.014338,0.049984,-0.021645,-0.003516,-0.011251,-0.024905,movie
'A' gai wak,-0.114455,0.049236,0.034111,0.006510,-0.012836,0.074500,0.064006,-0.022156,-0.054337,0.011865,...,0.089022,0.016201,0.042965,-0.108093,0.070876,0.028978,-0.021293,-0.020478,0.043706,movie
'Breaker' Morant,-0.114930,0.037942,-0.024508,-0.044298,0.029797,0.022180,0.019458,0.065945,0.001178,0.091680,...,0.035366,0.037543,-0.014330,-0.054029,0.015875,-0.022152,-0.068856,-0.022993,-0.009707,movie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
the sea inside,-0.040433,0.003638,0.014430,0.056221,0.036524,-0.064193,0.051530,-0.057853,-0.025589,0.028115,...,0.017674,-0.047550,0.037153,-0.103994,0.066041,0.063365,-0.026426,-0.088838,0.053737,book
a horse for angel,-0.051462,0.002771,0.040252,0.099846,-0.033508,0.006224,0.028362,-0.015769,0.106357,-0.060596,...,0.008437,0.029226,0.067755,0.012106,0.013726,-0.062873,0.034846,-0.031785,-0.072314,book
a faith worth sharing: a lifetime of conversations about christ,0.017713,-0.036133,-0.032067,0.048070,0.008477,0.058785,0.075231,-0.027118,0.041551,-0.058828,...,-0.025624,-0.028491,0.042647,-0.038489,-0.025794,0.021411,0.039490,-0.050125,-0.000608,book


In [ ]:
#PCA
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(n_components=2, svd_solver='auto')
pca_result = pca.fit_transform(df_.drop('label', axis=1))
#display(df)

fig = plt.figure(figsize=(14, 8))
x = list(pca_result[:,0])
y = list(pca_result[:,1])
# x and y given as array_like objects
import plotly.express as px
fig = px.scatter(df_, x=x, y=y, hover_name=df_.index, color=df_['label'])
fig.update_traces(textfont_size=14)
fig.show()

In [101]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd

nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(df_books_encoded.drop('label', axis=1))

In [ ]:
df_movies_raw.sample(10)

In [102]:
#input one movie
distances, indices = nbrs.kneighbors([df_movies_encoded.loc['The squid game'].drop('label', axis=0)])

#find 5 closest books
indices

#access book description
for a in indices[0]:
    print(df_books_raw.iloc[a].name)

the shift
nights
paperboy
ë‹¬ë¹› ì¡°ê°ì‚¬ 1
last call for the living


C:\Users\ardit\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


In [106]:
df_books_raw.loc['paperboy'].desc

"An 11-year-old boy living in Memphis in 1959 throws the meanest fastball in town, but talking is a whole different ball game. He can barely say a word without stuttering, not even his own name. So when he takes over his best friend's paper route for the month of July, he knows he'll be forced to communicate with the different customers, including a housewife who drinks too much and a retired merchant marine who seems to know just about everything. ,The paper route poses challenges, but it's a run-in with the neighborhood junkman, a bully and thief, that stirs up real trouble--and puts the boy's life, as well as that of his family's devoted housekeeper, in danger."

## find similar movie

In [22]:
import pandas as pd

df_movies_encoded = pd.read_csv('datasets/mpst_drop_encoded+.csv')
df_movies_encoded.index = df_movies_encoded.pop('title')
df_movies_encoded

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
title,,,,,,,,,,,,,,,,,,,,,
$,-0.024897,0.055245,-0.122327,-0.029603,-0.007433,0.036262,0.129328,0.020940,0.048176,-0.037479,...,0.032808,-0.081902,-0.002921,0.033075,0.036488,0.016339,0.081623,-0.065515,0.016297,-0.047112
$windle,-0.032736,0.183432,-0.103036,-0.038543,-0.011393,0.050009,0.066918,0.036443,0.018850,0.002050,...,0.012507,-0.092158,-0.028933,-0.058818,0.019628,-0.001966,0.089710,0.011076,-0.016599,0.075246
'71,-0.080760,0.002873,-0.099334,-0.058721,0.020339,0.047407,0.044057,-0.010748,-0.049804,0.035990,...,-0.013720,-0.032820,0.007075,0.006609,-0.014338,0.049984,-0.021645,-0.003516,-0.011251,-0.024905
'A' gai wak,-0.114455,0.049236,0.034111,0.006510,-0.012836,0.074500,0.064006,-0.022156,-0.054337,0.011865,...,-0.042240,0.089022,0.016201,0.042965,-0.108093,0.070876,0.028978,-0.021293,-0.020478,0.043706
'Breaker' Morant,-0.114930,0.037942,-0.024508,-0.044298,0.029797,0.022180,0.019458,0.065945,0.001178,0.091680,...,0.025499,0.035366,0.037543,-0.014330,-0.054029,0.015875,-0.022152,-0.068856,-0.022993,-0.009707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ådalen 31,-0.070786,0.102453,-0.025706,0.000993,0.014982,0.061297,0.107749,-0.013328,-0.007382,0.016728,...,0.006848,-0.029199,0.007583,-0.063745,-0.061957,0.053530,0.007582,0.002616,-0.015631,0.048152
Él,-0.032808,0.009943,0.047571,0.073746,0.043413,0.080470,-0.037416,0.024547,0.134191,-0.069619,...,0.038110,-0.026639,-0.023677,0.021118,0.026846,0.043454,-0.043832,0.004737,-0.037642,-0.059959
Ôdishon,-0.008681,-0.068630,-0.018248,0.010679,-0.005883,0.033397,0.035146,-0.007881,-0.012534,0.011920,...,-0.106348,-0.029262,-0.052140,-0.035167,0.040802,0.064893,0.135530,-0.004852,-0.008708,0.006679


In [35]:
from Levenshtein import distance
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd

nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(df_movies_encoded)

#string-searching algorithm
def closest_title(title):
    m = pd.DataFrame(df_movies_encoded.index)
    m['lev'] = m['title'].apply(lambda x : distance(x, 'Prince of Egypt'))
    return m.sort_values('lev', ascending=True)['title'].iloc[0]

def find_similar_movies(df, nbrs, title):
    #if title not in df it will choose the best search
    title = closest_title(title)

    distances, indices = nbrs.kneighbors([df.loc[title]])
    #print(indices)

    #we print df data, no longer df_
    for index in indices[0][1:]:
        print('index', index)
        print(title, '->', df.iloc[index].name)

find_similar_movies(df_movies_encoded, nbrs, 'Prince of Egypt')

index 11440
The Prince of Egypt -> The Ten Commandments: The Musical
